In [1]:
import pandas as pd
import numpy as np

In [68]:
data = np.array([1,2,3,4,5,6])
name = np.array(['' for x in range(6)])
besio = np.array(['' for x in range(6)])
entity = besio

In [69]:
columns = ['name/doi', 'data', 'BESIO', 'entity']

In [70]:
df = pd.DataFrame(np.array([name, data, besio, entity]).transpose(), columns=columns)

In [71]:
df.iloc[1,0] = 'doi'

In [97]:
hey = np.random.shuffle(data)

In [95]:
for piece in np.random.shuffle(data):
    print(piece)

TypeError: 'NoneType' object is not iterable

In [72]:
df

,name/doi,data,BESIO,entity
0,,1,,
1,doi,2,,
2,,3,,
3,,4,,
4,,5,,
5,,6,,


In [88]:
filename = 'carbon_ner_labels.xlsx'

In [93]:
append_df_to_excel(filename, df, startcol=0)
append_df_to_excel(filename, df, startcol=6)

In [91]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=0, startcol=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
                 
      startcol : upper left cell column to dump data frame.
                
                 
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    import pandas as pd

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        if startcol is None and sheet_name in writer.book.sheetnames:
            startcol = writer.book[sheet_name].max_col

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    
    if startcol is None:
        startcol = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, startcol=startcol, **to_excel_kwargs)

    # save the workbook
    writer.save()